# **PART 1**: Multi-fidelity Bayesian optimization (MFBO) with F3DASM

This Jupyter notebook contains the workflow to obtain the results presented in the mulfi-fidelity review manuscript.

## 0. Clone the correct version of F3DASM.
- Follow the procedure described on https://bessagroup.github.io/F3DASM/gettingstarted.html to access the `f3dasm` package.
- Clone F3DASM and select the `versionleo` branch: https://github.com/bessagroup/F3DASM/tree/versionleo

### 1. Import the necessary packages.

In [1]:
import f3dasm
import numpy as np

2022-11-22 13:27:45.534480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 13:27:45.667865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-22 13:27:45.667887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-22 13:27:45.701402: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-22 13:27:46.328110: W tensorflow/stream_executor/platform/de

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


### 2. Define the hyperparameters

Some hyperparameters are entered by default, for example the Gaussian process kernel and the multi-fidelity acquisition function.

In [2]:
dim = 1
iterations = 10
seed = 123
samp_nos = [20, 10]
fids = [0.5, 1.]
costs = [0.5, 1.]

### 3. Specify the multi-fidelity problem

- This means that the functions or models that represent the different fidelities need to be defined and collected into a multi-fidelity function. 

- The design space corresponding to this multi-fidelity function is also needed, in order to define a sampler to sample initial training data from.

The review manuscript works with augmented analytical functions to create the basis of a multi-fidelity problem. 

This starts by picking a base function.

In [3]:
base_fun = f3dasm.functions.AlpineN2(
    dimensionality=dim,
    scale_bounds=np.tile([0.0, 1.0], (dim, 1)),
    )

Once the function has been defined, the pipeline function `create_analytical_mf_problem` can be used to generate the multi-fidelity assets needed to run the optimization problem.

In [4]:

mf_train_data, mffun, _, mf_sampler = f3dasm.create_analytical_mf_problem(
    base_fun=base_fun,
    dim=dim,
    fids=fids,
    costs=costs,
    samp_nos=samp_nos,
    )

### 4. Define and initialize optimizer

This optimizer makes use of the variable-fidelity UCB acquisition function.

```
Jiang, P.; Cheng, J.; Zhou, Q.; Shu, L. & Hu, J.
Variable-fidelity lower confidence bounding approach for engineering optimization problems with expensive simulations
AIAA Journal, American Institute of Aeronautics and Astronautics, 2019, 57, 5416-5430
```

In [5]:
optimizer = f3dasm.optimization.MFBayesianOptimizationTorch(
    data=mf_train_data,
    mffun=mffun,
)

optimizer.init_parameters()

### 5. Run optimization and obtain results

In [6]:

res = f3dasm.run_mf_optimization(
    optimizer=optimizer,
    mffunction=mffun,
    sampler=mf_sampler[-1],
    iterations=iterations,
    seed=123,
    number_of_samples=samp_nos,
)


iteration 0
[[0.7905374 0.5      ]] [[-1.40396874]]
iteration 1
[[0.79067026 0.5       ]] [[-1.40398952]]
iteration 2
[[0.79076579 0.5       ]] [[-1.40400291]]
iteration 3
[[0.79074367 0.5       ]] [[-1.40399993]]
iteration 4
[[0.79086582 0.5       ]] [[-1.40401554]]
iteration 5
[[0.79088007 0.5       ]] [[-1.40401723]]
iteration 6
[[0.79089449 0.5       ]] [[-1.40401891]]
iteration 7
[[0.79092754 0.5       ]] [[-1.40402263]]
iteration 8
[[0.79082904 0.5       ]] [[-1.40401106]]
iteration 9
[[0.79097194 0.5       ]] [[-1.4040274]]


In [7]:

res[1].data

input         output
          x0  fid         y
0   0.000000  0.5  0.000000
1   0.500000  0.5  1.072110
2   0.750000  0.5 -1.284409
3   0.250000  0.5 -0.473134
4   0.375000  0.5  0.553412
5   0.875000  0.5 -0.923979
6   0.625000  0.5  0.041474
7   0.125000  0.5 -0.530499
8   0.187500  0.5 -0.653218
9   0.687500  0.5 -0.731371
10  0.937500  0.5 -0.076175
11  0.437500  0.5  0.986864
12  0.312500  0.5 -0.014665
13  0.812500  0.5 -1.373197
14  0.562500  0.5  0.725366
15  0.062500  0.5 -0.231280
16  0.093750  0.5 -0.390242
17  0.593750  0.5  0.412827
18  0.843750  0.5 -1.212044
19  0.343750  0.5  0.270328
20  0.000000  1.0  0.000000
21  0.500000  1.0  2.144220
22  0.750000  1.0 -2.568819
23  0.250000  1.0 -0.946268
24  0.375000  1.0  1.106824
25  0.875000  1.0 -1.847958
26  0.625000  1.0  0.082948
27  0.125000  1.0 -1.060997
28  0.187500  1.0 -1.306436
29  0.687500  1.0 -1.462742
30  0.790537  0.5 -1.403969
31  0.790670  0.5 -1.403990
32  0.790766  0.5 -1.404003
33  0.790744  0.5 -1.404000
34  0.790866  0.5 -1.404016
35  0.790880  0.5 -1.404017
36  0.790894  0.5 -1.404019
37  0.790928  0.5 -1.404023
38  0.790829  0.5 -1.404011
39  0.790972  0.5 -1.404027